In [ ]:
#%pip install --upgrade --quiet  langchain langchain-experimental langchain-openai neo4j wikipedia

#### Initiate environment and set example query

In [ ]:
import dotenv
import os
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer

In [ ]:
dotenv.load_dotenv()
DIFFBOT_API_KEY = os.environ['DIFFBOT_API_KEY']
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [ ]:
DIFFBOT_NLP = DiffbotGraphTransformer(diffbot_api_key=DIFFBOT_API_KEY)

In [ ]:
from langchain_community.document_loaders import WikipediaLoader

query = "Elon Musk"
raw_documents = WikipediaLoader(query=query).load()
graph_documents = DIFFBOT_NLP.convert_to_graph_documents(raw_documents)

In [ ]:
# observe raw doc retrieval from wiki
print(raw_documents)

#### Load a docker container to explore graph via GUI

In [ ]:
# get a docker going via CLI
# docker run \
#    --name neo4j \
#    -p 7474:7474 -p 7687:7687 \
#    -d \
#    -e NEO4J_AUTH=neo4j/pleaseletmein \
#    -e NEO4J_PLUGINS=\[\"apoc\"\]  \
#    neo4j:latest

In [ ]:
from langchain_community.graphs import Neo4jGraph

# if running docker in WSL, command wsl hostname -I to get WSL IP
# neo4j gui port is 7474
url = "bolt://172.29.153.143:7687"
username = "neo4j"
password = "pleaseletmein"

graph = Neo4jGraph(url=url, username=username, password=password)

Load GraphDocuments into the docker knowledge graph then refresh graph schema

In [ ]:
graph.add_graph_documents(graph_documents)

In [ ]:
graph.refresh_schema()

In [ ]:
print(graph.schema)

In [ ]:
# test a graph query manually to verify connectivity before burning OpenAI calls
graph.query(
    """
MATCH p=()-[r:CHIEF_EXECUTIVE_OFFICER]->() RETURN p LIMIT 25
"""
)

Log into Neo4J by navigating to http://\{docker ip}/7474 and logging in with credentials set above. To view the graph, you can run a simple query of `MATCH (n) RETURN n LIMIT 25`

#### Query the graph

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(temperature=0, model_name="gpt-4-0125-preview"),
    qa_llm=ChatOpenAI(temperature=.3, model_name="gpt-3.5-turbo"),
    graph=graph,
    verbose=True,
)

In [ ]:
chain.run("Does Elon Musk have kids?")
